In [890]:
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [891]:
import requests
import json
import pandas as pd

# Complete dataspace flow

## Consts


In [892]:
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"
api_endpoint = "https://www.publico.es"

data_space_provider_participant_id = ""
data_space_consumer_participant_id = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

## Participants onboarding

In [893]:
url = data_space_provider + "/api/v1/participants"
payload = {
    "participantType": "Provider",
    "participantBaseUrl": data_space_provider,
    "extraFields": {}
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["participant_id"]
data_space_provider_participant_id


'urn:uuid:e4619281-4a88-452d-a74d-17e881512845'

In [894]:
url = data_space_provider + "/api/v1/participants"
payload = {
    "participantType": "Consumer",
    "participantBaseUrl": data_space_consumer,
    "extraFields": {
        "did": "hola"
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json["participant_id"]
data_space_consumer_participant_id

'urn:uuid:f69202ab-ca60-4a13-b9bd-d750a23fdd98'

## Create Catalog instances and metadata

In [895]:
url = data_space_provider + "/api/v1/catalogs"
payload = {
    "foaf:homepage": "My dataspace catalog",
    "dct:title": "My dataspace catalog",

}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]
catalog_id

'urn:uuid:ae0e4cd7-79d2-49de-a55e-353acc28f184'

In [896]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets".format(catalog_id)
payload = {
    "dct:title": "My dataspace dataset",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
dataset_id

'urn:uuid:0cd8402c-8600-4360-b1c2-ccf067dc288e'

In [897]:
url = data_space_provider + "/api/v1/catalogs/{}/data-services".format(catalog_id)
payload = {
    "dcat:endpointURL": api_endpoint
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_service_id = response_as_json["@id"]
data_service_id

'urn:uuid:a4916d40-9283-469e-8e17-a998cd84f9b1'

In [898]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets/{}/distributions".format(catalog_id, dataset_id)
payload = {
    "dcat:accessService": data_service_id,
    "dct:formats": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["@id"]
distribution_id

'urn:uuid:9dc2fce6-a5d2-43fc-9d62-70eed9760e17'

## Create policies

In [899]:
url = data_space_provider + "/api/v1/datasets/{}/policies".format(dataset_id)
payload = {
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }

    ],
    "obligation": [],
    "prohibition": []
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["@id"]
target_id = response_as_json["target"]
print(json.dumps(response_as_json, indent=4))

{
    "@id": "urn:uuid:78fdd8b5-7290-47bd-a0d3-a0b8640a256e",
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }
    ],
    "obligation": [],
    "@type": "Offer",
    "prohibition": [],
    "target": "urn:uuid:0cd8402c-8600-4360-b1c2-ccf067dc288e"
}


## Contract Negotiation

In [900]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerAddress": data_space_provider,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
  "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
  "offer": {
    "@id": "urn:uuid:78fdd8b5-7290-47bd-a0d3-a0b8640a256e",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:0cd8402c-8600-4360-b1c2-ccf067dc288e"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
    "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
    "state": "REQUESTED"
  }
}


In [901]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id, # remove to test offer from provider
    "providerPid": cn_provider_id, # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "urn:uuid:f69202ab-ca60-4a13-b9bd-d750a23fdd98",
  "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
  "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
  "offer": {
    "@id": "urn:uuid:78fdd8b5-7290-47bd-a0d3-a0b8640a256e",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:0cd8402c-8600-4360-b1c2-ccf067dc288e"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
    "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
    "state": "OFFERED"
  }
}


In [902]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerAddress": data_space_provider,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
  "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
  "offer": {
    "@id": "urn:uuid:78fdd8b5-7290-47bd-a0d3-a0b8640a256e",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:0cd8402c-8600-4360-b1c2-ccf067dc288e"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
    "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
    "state": "REQUESTED"
  }
}


In [903]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "urn:uuid:f69202ab-ca60-4a13-b9bd-d750a23fdd98",
  "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
  "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
  "offer": {
    "@id": "urn:uuid:78fdd8b5-7290-47bd-a0d3-a0b8640a256e",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:0cd8402c-8600-4360-b1c2-ccf067dc288e"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
    "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
    "state": "OFFERED"
  }
}


In [904]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-acceptance"
payload = {
    "providerAddress": data_space_provider,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
  "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
    "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
    "state": "ACCEPTED"
  }
}


In [905]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-agreement"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
agreement_id=response_as_json["agreement"]["@id"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
  "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
  "agreement": {
    "@id": "urn:uuid:3e43d4c9-8a75-4acc-92e3-f7c7fed92703",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Agreement",
    "target": "urn:uuid:0cd8402c-8600-4360-b1c2-ccf067dc288e",
    "assigner": "urn:uuid:e4619281-4a88-452d-a74d-17e881512845",
    "assignee": "urn:uuid:f69202ab-ca60-4a13-b9bd-d750a23fdd98",
    "timestamp": "2025-06-05T12:53:59.853709+00:00"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
    "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
    "state": "AGREED"
  }
}


In [906]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-verification"
payload = {
    "providerAddress": data_space_provider,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
  "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
    "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
    "state": "VERIFIED"
  }
}


In [907]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-finalization"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
  "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:8c404b17-16fb-411b-905a-ed53e8e57e85",
    "consumerPid": "urn:uuid:cb557a0f-7f24-4d17-b6f1-19f3b3ab5dcb",
    "state": "FINALIZED"
  }
}


## Transfer Negotiation


In [908]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-request"
payload = {
    "providerAddress": data_space_provider,
    "agreementId": agreement_id,
    "format": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_consumer_id = response_as_json["consumerPid"]
transfer_provider_id = response_as_json["providerPid"]
callback_address = response_as_json["callbackAddress"]
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:827ff16c-9b17-4f00-9ed8-bb162fee1dc0",
  "consumerPid": "urn:uuid:fba48cf2-8140-4fec-bef7-1f323aeb7c3a",
  "agreementId": "urn:uuid:3e43d4c9-8a75-4acc-92e3-f7c7fed92703",
  "callbackAddress": "http://127.0.0.1:1100/urn:uuid:a8b9d64a-0e4c-4e9b-98a2-822f94dc53a5",
  "format": "http+pull",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:827ff16c-9b17-4f00-9ed8-bb162fee1dc0",
    "consumerPid": "urn:uuid:fba48cf2-8140-4fec-bef7-1f323aeb7c3a",
    "state": "REQUESTED"
  }
}


In [909]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-start"
payload = {
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:827ff16c-9b17-4f00-9ed8-bb162fee1dc0",
  "consumerPid": "urn:uuid:fba48cf2-8140-4fec-bef7-1f323aeb7c3a",
  "dataAddress": {
    "@type": "DataAddress",
    "endpointType": "http",
    "endpoint": "http://127.0.0.1:1200/data/urn:uuid:827ff16c-9b17-4f00-9ed8-bb162fee1dc0",
    "endpointProperties": [
      {
        "@type": "EndpointProperty",
        "name": "authType",
        "value": ""
      },
      {
        "@type": "EndpointProperty",
        "name": "authorization",
        "value": ""
      }
    ]
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:827ff16c-9b17-4f00-9ed8-bb162fee1dc0",
    "consumerPid": "urn:uuid:fba48cf2-8140-4fec-bef7-1f323aeb7c3a",
    "state": "STARTED"
  }
}


In [910]:
url = data_space_consumer + "/api/v1/dataplane/" + transfer_consumer_id
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
consumer_data_address = response_as_json["process_address"]["url"]
print(json.dumps(response_as_json, indent=2))

{
  "id": "urn:uuid:fba48cf2-8140-4fec-bef7-1f323aeb7c3a",
  "process_direction": "PULL",
  "upstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "downstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "process_address": {
    "protocol": "http",
    "url": "http://127.0.0.1:1100/data/urn:uuid:fba48cf2-8140-4fec-bef7-1f323aeb7c3a",
    "auth_type": "",
    "auth_content": ""
  },
  "created_at": "2025-06-05T12:54:00.253306",
  "updated_at": "2025-06-05T12:54:00.290469",
  "state": "STARTED"
}


In [911]:
df = pd.read_json(consumer_data_address)
df

ValueError: Expected object or value

In [749]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "providerAddress": data_space_provider,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:5cab7f0a-780b-4859-8aed-13cc86bec33f",
  "consumerPid": "urn:uuid:114cda37-27d5-494a-af8a-5f36a02dec28",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:5cab7f0a-780b-4859-8aed-13cc86bec33f",
    "consumerPid": "urn:uuid:114cda37-27d5-494a-af8a-5f36a02dec28",
    "state": "SUSPENDED"
  }
}


In [742]:
df = pd.read_json(consumer_data_address)
df

HTTPError: HTTP Error 403: Forbidden

In [750]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-start"
payload = {
    "providerAddress": data_space_provider,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:5cab7f0a-780b-4859-8aed-13cc86bec33f",
  "consumerPid": "urn:uuid:114cda37-27d5-494a-af8a-5f36a02dec28",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:5cab7f0a-780b-4859-8aed-13cc86bec33f",
    "consumerPid": "urn:uuid:114cda37-27d5-494a-af8a-5f36a02dec28",
    "state": "STARTED"
  }
}


In [744]:
df = pd.read_json(consumer_data_address)
df

,postId,id,name,email,body
0,1,1,id labore ex et quam laborum,Eliseo@gardner.biz,laudantium enim quasi est quidem magnam volupt...
1,1,2,quo vero reiciendis velit similique earum,Jayne_Kuhic@sydney.com,est natus enim nihil est dolore omnis voluptat...
2,1,3,odio adipisci rerum aut animi,Nikita@garfield.biz,quia molestiae reprehenderit quasi aspernatur\...
3,1,4,alias odio sit,Lew@alysha.tv,non et atque\noccaecati deserunt quas accusant...
4,1,5,vero eaque aliquid doloribus et culpa,Hayden@althea.biz,harum non quasi et ratione\ntempore iure ex vo...
...,...,...,...,...,...
495,100,496,et occaecati asperiores quas voluptas ipsam no...,Zola@lizzie.com,neque unde voluptatem iure\nodio excepturi ips...
496,100,497,doloribus dolores ut dolores occaecati,Dolly@mandy.co.uk,non dolor consequatur\nlaboriosam ut deserunt ...
497,100,498,dolores minus aut libero,Davion@eldora.net,aliquam pariatur suscipit fugiat eos sunt\nopt...
498,100,499,excepturi sunt cum a et rerum quo voluptatibus...,Wilburn_Labadie@araceli.name,et necessitatibus tempora ipsum quaerat invent...


In [745]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:f7d9bd22-6b7c-485e-ab36-ebac0e9f7a19",
  "consumerPid": "urn:uuid:bce73f8f-43b4-434c-8c71-062f5cfb6264",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:f7d9bd22-6b7c-485e-ab36-ebac0e9f7a19",
    "consumerPid": "urn:uuid:bce73f8f-43b4-434c-8c71-062f5cfb6264",
    "state": "SUSPENDED"
  }
}


In [746]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-completion"
payload = {
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:f7d9bd22-6b7c-485e-ab36-ebac0e9f7a19",
  "consumerPid": "urn:uuid:bce73f8f-43b4-434c-8c71-062f5cfb6264",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:f7d9bd22-6b7c-485e-ab36-ebac0e9f7a19",
    "consumerPid": "urn:uuid:bce73f8f-43b4-434c-8c71-062f5cfb6264",
    "state": "COMPLETED"
  }
}
